# [CDAF] Atividade 5

## Nome
Nome: RODRIGO FELIPE LIMA BRAZ


## Referências
- [1] https://tomdecroos.github.io/reports/kdd19_tomd.pdf
- [2] https://socceraction.readthedocs.io/en/latest/api/vaep.html
- [3] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/vaep.html
- [4] https://github.com/ML-KULeuven/socceraction/tree/master/public-notebooks

## Introdução
- Nessa atividade, temos implementada a pipeline inteira do VAEP [1] para os dados do Wyscout das Top 5 ligas.
- [2] é a documentação das funções do VAEP na API do socceraction.
- [3] apresenta uma explicação do framework com uma mistura de intuição, matemática e código.
- [4] são notebooks públicos que implementam o VAEP para outro conjunto de dados.

## Instruções
- Para cada header do notebook abaixo, vocês devem explicar o que foi feito e à qual seção/subseção/equação do paper "Actions Speak Louder than Goals: Valuing Actions by Estimating Probabilities" ela corresponde. Justifique suas respostas.
- Além disso, após algumas partes do código haverão perguntas que vocês devem responder, possivelmente explorando minimamente o que já está pronto.
- Por fim, vocês devem montar um diagrama do fluxo de funções/tarefas de toda a pipeline do VAEP abaixo. Esse diagrama deve ser enviado como arquivo na submissão do Moodle, para além deste notebook.

### Carregando os dados

In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_matches(path):
    matches = pd.read_json(path_or_buf=path)
    # as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
    team_matches = []
    for i in range(len(matches)):
        match = pd.DataFrame(matches.loc[i, 'teamsData']).T
        match['matchId'] = matches.loc[i, 'wyId']
        team_matches.append(match)
    team_matches = pd.concat(team_matches).reset_index(drop=True)

    return team_matches

In [3]:
def load_players(path):
    players = pd.read_json(path_or_buf=path)
    players['player_name'] = players['firstName'] + ' ' + players['lastName']
    players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})

    return players

In [4]:
def load_events(path):
    events = pd.read_json(path_or_buf=path)
    # pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
    events = events.rename(columns={
        'id': 'event_id',
        'eventId': 'type_id',
        'subEventId': 'subtype_id',
        'teamId': 'team_id',
        'playerId': 'player_id',
        'matchId': 'game_id'
    })
    events['milliseconds'] = events['eventSec'] * 1000
    events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

    return events

In [5]:
def load_minutes_played_per_game(path):
    minutes = pd.read_json(path_or_buf=path)
    minutes = minutes.rename(columns={
        'playerId': 'player_id',
        'matchId': 'game_id',
        'teamId': 'team_id',
        'minutesPlayed': 'minutes_played'
    })
    minutes = minutes.drop(['shortName', 'teamName', 'red_card'], axis=1)

    return minutes

In [6]:
leagues = ['England', 'Spain']
events = {}
matches = {}
minutes = {}
for league in leagues:
    path = r'C:\Users\rodrigo.braz\Rodrigo_Si\pratices\CDAF\ATV5\matches_{}.json'.format(league)
    matches[league] = load_matches(path)
    path = r'C:\Users\rodrigo.braz\Rodrigo_Si\pratices\CDAF\ATV5\events_{}.json'.format(league)
    events[league] = load_events(path)
    path = r'C:\Users\rodrigo.braz\Rodrigo_Si\pratices\CDAF\ATV5\minutes_played_per_game_{}.json'.format(league)
    minutes[league] = load_minutes_played_per_game(path)

In [7]:
path = r"C:\Users\rodrigo.braz\Rodrigo_Si\pratices\CDAF\ATV5\players.json"
players = load_players(path)
players['player_name'] = players['player_name'].str.decode('unicode-escape')

### SPADL

In [8]:
from tqdm import tqdm
import socceraction.spadl as spd

In [9]:
def spadl_transform(events, matches):
    spadl = []
    game_ids = events.game_id.unique().tolist()
    for g in tqdm(game_ids):
        match_events = events.loc[events.game_id == g]
        match_home_id = matches.loc[(matches.matchId == g) & (matches.side == 'home'), 'teamId'].values[0]
        match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
        match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
        match_actions = spd.add_names(match_actions)
        spadl.append(match_actions)
    spadl = pd.concat(spadl).reset_index(drop=True)

    return spadl

In [10]:
spadl = {}
for league in leagues:
    spadl[league] = spadl_transform(events=events[league], matches=matches[league])

100%|██████████| 380/380 [03:30<00:00,  1.80it/s]


Neste header os dados foram carregados e transformados para o padrão spald, para ficar mais fácil fazer as análises e utilização de features.


### Features

In [11]:
from socceraction.vaep import features as ft

In [12]:
def features_transform(spadl):
    spadl.loc[spadl.result_id.isin([2, 3]), ['result_id']] = 0
    spadl.loc[spadl.result_name.isin(['offside', 'owngoal']), ['result_name']] = 'fail'

    xfns = [
        ft.actiontype_onehot,
        ft.bodypart_onehot,
        ft.result_onehot,
        ft.goalscore,
        ft.startlocation,
        ft.endlocation,
        ft.team,
        ft.time,
        ft.time_delta
    ]

    features = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        match_states = ft.gamestates(actions=match_actions)
        match_feats = pd.concat([fn(match_states) for fn in xfns], axis=1)
        features.append(match_feats)
    features = pd.concat(features).reset_index(drop=True)

    return features

Header que visa escolher as features que serão utilizadas no modelo.

1- O que a primeira e a segunda linhas da função acima fazem? Qual sua hipótese sobre intuito dessas transformações? Como você acha que isso pode impactar o modelo final?

A primeira está trocando tudo que for result = 2 ou 3 para 1, para deixar apenas sucesso ou falha no resultado da ação.
A segunda está setando ações que estão impedidos, ou gols contra como ações falhas. Acredito que essas transformações deixem o modelo mais determinístico, uma vez que você caracteriza ações apenas como sucesso ou falha.

In [13]:
features = {}
for league in ['England', 'Spain']:
    features[league] = features_transform(spadl[league])

100%|██████████| 380/380 [00:17<00:00, 21.44it/s]


In [14]:
features

{'England':         type_pass_a0  type_cross_a0  type_throw_in_a0  \
 0               True          False             False   
 1               True          False             False   
 2               True          False             False   
 3               True          False             False   
 4               True          False             False   
 ...              ...            ...               ...   
 482896          True          False             False   
 482897         False           True             False   
 482898         False          False             False   
 482899         False          False             False   
 482900         False          False             False   
 
         type_freekick_crossed_a0  type_freekick_short_a0  \
 0                          False                   False   
 1                          False                   False   
 2                          False                   False   
 3                          False              

### Labels

In [15]:
import socceraction.vaep.labels as lab

In [16]:
def labels_transform(spadl):
    yfns = [lab.scores, lab.concedes]

    labels = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        labels.append(pd.concat([fn(actions=match_actions) for fn in yfns], axis=1))

    labels = pd.concat(labels).reset_index(drop=True)

    return labels

In [17]:
labels = {}
for league in ['England', 'Spain']:
    labels[league] = labels_transform(spadl[league])

100%|██████████| 380/380 [00:19<00:00, 19.23it/s]


In [18]:
labels

{'England':         scores  concedes
 0        False     False
 1        False     False
 2        False     False
 3        False     False
 4        False     False
 ...        ...       ...
 482896   False     False
 482897   False     False
 482898   False     False
 482899   False     False
 482900   False     False
 
 [482901 rows x 2 columns],
 'Spain':         scores  concedes
 0        False     False
 1        False     False
 2        False     False
 3        False     False
 4        False     False
 ...        ...       ...
 473889   False     False
 473890   False     False
 473891   False     False
 473892   False     False
 473893   False     False
 
 [473894 rows x 2 columns]}

In [19]:
labels['England']['scores'].sum()

7553

In [20]:
labels['England']['concedes'].sum()

2313

Etapa que instância as labels a serem usadas no modelo.

2- Explique o por que da quantidade de labels positivos do tipo scores ser muito maior que do concedes. Como você acha que isso pode impactar o modelo final? 

Pois o modelo visa medir a contribuição do jogador na construção das jogadas do time, logo, a quantidade maior da label scores já é esperado dada a natureza da operação que VAEP mede.

### Training Model

In [21]:
import xgboost as xgb
import sklearn.metrics as mt

In [22]:
def train_vaep(X_train, y_train, X_test, y_test):
    models = {}
    for m in ['scores', 'concedes']:
        models[m] = xgb.XGBClassifier(random_state=0, n_estimators=50, max_depth=3)

        print('training ' + m + ' model')
        models[m].fit(X_train, y_train[m])

        p = sum(y_train[m]) / len(y_train[m])
        base = [p] * len(y_train[m])
        y_train_pred = models[m].predict_proba(X_train)[:, 1]
        train_brier = mt.brier_score_loss(y_train[m], y_train_pred) / mt.brier_score_loss(y_train[m], base)
        print(m + ' Train NBS: ' + str(train_brier))
        print()

        p = sum(y_test[m]) / len(y_test[m])
        base = [p] * len(y_test[m])
        y_test_pred = models[m].predict_proba(X_test)[:, 1]
        test_brier = mt.brier_score_loss(y_test[m], y_test_pred) / mt.brier_score_loss(y_test[m], base)
        print(m + ' Test NBS: ' + str(test_brier))
        print()

        print('----------------------------------------')

    return models

In [23]:
models = train_vaep(X_train=features['England'], y_train=labels['England'], X_test=features['Spain'], y_test=labels['Spain'])

training scores model
scores Train NBS: 0.8452154331682665

scores Test NBS: 0.8503669232527757

----------------------------------------
training concedes model
concedes Train NBS: 0.9644632156409795

concedes Test NBS: 0.9745272571693011

----------------------------------------


Objetivo desse header era treinar modelos que vão ser úteis para calcular o valor de ações nos próximos headers.


3- Por que treinamos dois modelos diferentes? Por que a performance dos dois é diferente?


Você treina com os dados em uma liga e testa em outra pra ver o comportamento do modelo em uma liga a qual ele não foi mapeado. Isso pode ser muito útil pra evitar um pouco do vies.

### Predictions

In [24]:
def generate_predictions(features, models):
    preds = {}
    for m in ['scores', 'concedes']:
        preds[m] = models[m].predict_proba(features)[:, 1]
    preds = pd.DataFrame(preds)

    return preds

In [39]:
preds = {}
preds['Spain'] = generate_predictions(features=features['Spain'], models=models)
preds

{'Spain':           scores  concedes
 0       0.004560  0.000367
 1       0.003573  0.000347
 2       0.002895  0.000345
 3       0.002162  0.000318
 4       0.002424  0.001799
 ...          ...       ...
 473889  0.033276  0.002812
 473890  0.041886  0.002787
 473891  0.017484  0.004722
 473892  0.007541  0.012254
 473893  0.005007  0.047561
 
 [473894 rows x 2 columns]}

### Action Values

In [26]:
import socceraction.vaep.formula as fm

In [27]:
def calculate_action_values(spadl, predictions):
    action_values = fm.value(actions=spadl, Pscores=predictions['scores'], Pconcedes=predictions['concedes'])
    action_values = pd.concat([
        spadl[['player_id', 'original_event_id', 'action_id', 'game_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_name', 'result_name']],
        predictions.rename(columns={'scores': 'Pscores', 'concedes': 'Pconcedes'}),
        action_values
    ], axis=1)

    return action_values

In [52]:
action_values = {}
action_values['Spain'] = calculate_action_values(spadl=spadl['Spain'], predictions=preds['Spain'])
df = action_values['Spain']
df['Pscores'] = df['Pscores'].astype(float)
filt = df['Pscores'] > 0.95
filt2 = (df['type_name'] == 'shot') & (df['result_name'] == 'fail')
df[filt2]

,player_id,original_event_id,action_id,game_id,start_x,start_y,end_x,end_y,type_name,result_name,Pscores,Pconcedes,offensive_value,defensive_value,vaep_value
20,225089,180865315,20,2565548,97.65,44.88,105.00,34.00,shot,fail,0.031424,0.003259,-0.048751,-0.002252,-0.051003
22,255738,180864547,22,2565548,84.00,27.88,84.00,27.88,shot,fail,0.012689,0.002681,-0.012616,0.000804,-0.011812
93,37831,180864486,93,2565548,92.40,29.24,92.40,29.24,shot,fail,0.026429,0.003202,-0.033228,-0.001799,-0.035026
96,15214,180864491,96,2565548,91.35,23.12,105.00,27.20,shot,fail,0.025181,0.005027,-0.030991,-0.002125,-0.033117
178,225089,180864792,178,2565548,78.75,40.80,105.00,34.00,shot,fail,0.023212,0.004351,0.004753,-0.001096,0.003657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473619,3321,253302307,1212,2565927,92.40,20.40,105.00,34.00,shot,fail,0.053304,0.003787,-0.013934,-0.002090,-0.016024
473633,267134,253302272,1226,2565927,97.65,23.12,97.65,23.12,shot,fail,0.033856,0.006212,-0.049710,-0.002961,-0.052671
473673,267134,253302329,1266,2565927,94.50,46.24,94.50,46.24,shot,fail,0.027493,0.005142,-0.038734,-0.001858,-0.040592
473851,3321,253302642,1444,2565927,86.10,47.60,105.00,30.60,shot,fail,0.045398,0.006529,-0.009160,-0.003653,-0.012813


4- Explore as ações com Pscores >= 0.95. Por que elas tem um valor tão alto? As compare com ações do mesmo tipo e resultado opostado. Será que o modelo aprende que essa combinação de tipo de ação e resultado está diretamente relacionado à variável y que estamos tentando prever?

5- Qual formula do paper corresponde à coluna 'offensive_value' do dataframe action_values? E a coluna 'defensive_value'?

4 - A maior parte das ações que possuem um action value > 0.95 são chutes com success = true, o que leva a gente imaginar que o modelo beneficia aqueles jogadores que dão o último toque que seria o chute e tem sucesso, idependentemente de posição do chute ou coisas desse tipo, já que os chutes com success = false tem um actionvalue imensamente menores.

5

Ofensive: Scores
Defensive: concedes

O objetivo desse header era calcular, com base no modelo treinado, os valores de cada ação de cada jogador.

### Player Ratings

In [ ]:
def calculate_minutes_per_season(minutes_per_game):
    minutes_per_season = minutes_per_game.groupby('player_id', as_index=False)['minutes_played'].sum()

    return minutes_per_season

In [ ]:
minutes_per_season = {}
minutes_per_season['Spain'] = calculate_minutes_per_season(minutes['Spain'])

In [ ]:
minutes_per_season

{'Spain':      player_id  minutes_played
 0           33              93
 1           99             103
 2          151            1379
 3          254            3230
 4          786              83
 ..         ...             ...
 552     519496              65
 553     520163             124
 554     545811              24
 555     551398              63
 556     568583              10
 
 [557 rows x 2 columns]}

In [ ]:
def calculate_player_ratings(action_values, minutes_per_season, players):
    player_ratings = action_values.groupby(by='player_id', as_index=False).agg({'vaep_value': 'sum'}).rename(columns={'vaep_value': 'vaep_total'})
    player_ratings = player_ratings.merge(minutes_per_season, on=['player_id'], how='left')
    player_ratings['vaep_p90'] = player_ratings['vaep_total'] / player_ratings['minutes_played'] * 90
    player_ratings = player_ratings[player_ratings['minutes_played'] >= 600].sort_values(by='vaep_p90', ascending=False).reset_index(drop=True)
    player_ratings = player_ratings.merge(players, on=['player_id'], how='left')
    player_ratings = player_ratings[['player_id', 'player_name', 'minutes_played', 'vaep_total', 'vaep_p90']]

    return player_ratings

In [ ]:
player_ratings = {}
player_ratings['Spain'] = calculate_player_ratings(action_values=action_values['Spain'], minutes_per_season=minutes_per_season['Spain'], players=players)

In [ ]:
player_ratings

{'Spain':      player_id                          player_name  minutes_played  \
 0         3359       Lionel Andrés Messi Cuccittini          3108.0   
 1         8278                    Gareth Frank Bale          1850.0   
 2         3802            Philippe Coutinho Correia          1329.0   
 3         3322  Cristiano Ronaldo dos Santos Aveiro          2355.0   
 4         3682                    Antoine Griezmann          2591.0   
 ..         ...                                  ...             ...   
 386     105616                     Brown Ideye Aide           764.0   
 387       5911             Sandro  Ramírez Castillo           747.0   
 388     228902                     Jonathan Calleri          3210.0   
 389       3875                 Borja González Tomás          1029.0   
 390       6914             Alexander Alegría Moreno           647.0   
 
      vaep_total  vaep_p90  
 0     35.891376  1.039326  
 1     14.323647  0.696826  
 2     10.036554  0.679676  
 3     17

Objetivo do header era fazer o groupby das action values com base no modelo vaep para cada jogador, e rankear esses jogadores, além de levar em conta action_values / minutos jogados

6- Acha que o Top 5 da lista é bem representativo? Compare esse ranqueamento do VAEP com o do xT da Atividade 4. Qual você acha que é mais representativo?

Acredito que qualquer modelo tenha o ponto central do que ele quer medir. Em contraponto, existem jogadores que são outliers para qualquer caso quando se usa caracteristicas que esses jogadores possuem, independente do que se quer medir. Ou seja, como esses modelos não querem medir necessariamente ações defensivas, tem jogadores tal como Lionel Messi que é o TOP1 da lista, que sempre vão se destacar pois são jogadores fora da curva. Logo, acho que o top5 é bem representativo, é de se esperar que mesmo ranqueamento do VAEP quanto do xT vão mapear esses outliers que é por exemplo Lionel Messi como jogador de futebol. Ademais, é inegável que o modelo VAEP e o xT, apesar de similaridades em alguns pontos, visam medir parâmetros diferentes de ações jogadores. No xT, era muito mais comum aparição de defensores, tal como apareceu Marcelo em segundo lugar na atividade4, ou seja, são modelos diferentes que entregam resultados diferentes, pois possuem premissas e objetivos diferentes apesar a similaridade de algumas características.


Não entendi muito bem a parte do diagrama, por isso vou tentar resumir:

Extração dos dados > tratamento dos dados > transformações para o padrão SPADL> escolha e criação das features > instanciação das labels > treino dos modelos> uso do modelo para avaliar o valor das ações > agregação das ações por jogador > divisão pela quantidade de minutos jogados do jogador.